In [1]:
import numpy as np
import matplotlib.pyplot as plt
from cartpole_lib_python import *
import odrive

In [2]:
odrv0 = odrive.find_any()

In [3]:
init_odrive()

axis0
  active_errors: no error
  disarm_reason: no error
  procedure_result: ProcedureResult.SUCCESS
  last_drv_fault: none
internal issues: none
axis0
  active_errors: no error
  disarm_reason: no error
  procedure_result: ProcedureResult.SUCCESS
  last_drv_fault: none
internal issues: none
axis0
  active_errors: no error
  disarm_reason: no error
  procedure_result: ProcedureResult.SUCCESS
  last_drv_fault: none
internal issues: none
odrive initialized


In [6]:
odrv = get_odrive()
odrive.utils.dump_errors(odrv)

axis0
  active_errors: no error
  disarm_reason: no error
  procedure_result: ProcedureResult.SUCCESS
  last_drv_fault: none
internal issues: none


In [7]:
def busy_sleep1(duration, loop_start, get_now=time.time):
    # now = time.perf_counter()
    now = get_now()

    end = duration + loop_start

    while now < end:
        # now = time.perf_counter()
        now = get_now()


In [132]:
print(get_state())

[0.02421487603305785, 1.5447186534008406, -8.837207472556526e-83, -0.0001306370542450086]


1.5447186534008406

In [ ]:
control_frequency = 30 # Hz
control_period = 1/control_frequency
T = 2 # runtime, seconds
N = int(T*control_frequency)
print("N: ", N)

k_matrix = np.array([-4.68901, -27.56145, -4.56255, -5.56030]) # 30hz

control_clamp = 0.1
xg = np.array([0, 1.5447186534008406, 0, 0])


# 30 hz
A = np.array([
 [1.0,  -0.00263967,  0.0333333,  -2.92458e-5],
 [0.0,   1.01727,     0.0,         0.0335247],
 [0.0,  -0.158834,    1.0,        -0.00263967],
 [0.0,   1.03916,     0.0,         1.01727]])

B = np.array([
 [ 0.001166386647061344],
 [-0.0024900041079065627],
 [ 0.07004855111703442],
 [-0.1498278085801933]])


H = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0]])

meas_noise_cov = np.diag([.1,.1]) # most recent
process_noise_cov = np.diag([1,1,50,50]) # position, angle, linear velocity, angular velocity

nx = process_noise_cov.shape[0]
def kf_update(x_meas, x_pred_prev, est_cov_prev):
    Kn = est_cov_prev@H.T @ np.linalg.inv(H@est_cov_prev@H.T + meas_noise_cov)
    x = x_pred_prev + Kn @ (x_meas - H@x_pred_prev)
    est_cov = (np.eye(nx) - Kn@H) @ est_cov_prev @ (np.eye(nx) - Kn@H).T + Kn@meas_noise_cov@Kn.T
    return x, est_cov

def kf_predict(state, control, est_cov):
    x_pred = A@state + B@control
    est_cov_pred = A@est_cov@A.T + process_noise_cov
    return x_pred, est_cov_pred

# Initialize Kalman filter variables
# First state estimate
meas = get_measurement() - xg[:2]
x = np.array([meas[0], meas[1], 0, 0])
x_pred_prev = x
# Estimation covariance
est_cov = np.eye(4)*20
est_cov_prev = est_cov

for i in range(N):
    time_start = time.perf_counter()
    
    measurement = get_measurement() - xg[:2]
    if i > 0:
        x, est_cov = kf_update(measurement, x_pred_prev, est_cov_prev)
    
    x[3] /= 2
    control = -k_matrix @ x * 0.2
    control = np.min(control_clamp, np.max(-control_clamp, control))
    
    x_pred_prev, est_cov_prev = kf_predict(x, [control], est_cov)
    
    command_linear_torque(-control)